#Import ImageDataGenerator Library and configure it

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline


In [2]:
import abc
import functools
import multiprocessing
import sys
import threading
import warnings

import numpy as np
import six

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 
test_datagen=ImageDataGenerator(rescale = 1./255) 

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GMKt23iW2XdnrnutVPY4alkQgbgl5DYr-wBW2LYLge42',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-omd0vkwic10b6g'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
pwd

'/home/wsuser/work'

In [7]:
import os
filenamer = os.listdir('/home/wsuser/work/Dataset/training_set')

In [8]:
dir1='/home/wsuser/work/Dataset/training_set'

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
xtrain=train_datagen.flow_from_directory("/home/wsuser/work/Dataset/training_set",target_size=(64,64),class_mode="categorical",batch_size=25)

Found 15750 images belonging to 9 classes.


In [12]:
xtest=test_datagen.flow_from_directory("/home/wsuser/work/Dataset/test_set",target_size=(64,64),
class_mode='categorical' , batch_size=25)

Found 2250 images belonging to 9 classes.


In [13]:
xtrain.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [14]:
xtest.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [70]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [16]:
model=Sequential()

In [17]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))

In [18]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [19]:
model.add(Flatten())

In [20]:
model.add(Dense (units=512, activation='relu')) 
model.add(Dense (units=9, activation='softmax'))

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
len(xtrain)

630

In [23]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [24]:
early_stopping = EarlyStopping(monitor='val_accuracy',
                        patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                        patience=5,
                        factor=0.5,min_lr=0.00001)

callback = [reduce_lr,early_stopping]

In [25]:
model.fit(xtrain,
            steps_per_epoch=len(xtrain),
            epochs=10,
            callbacks=callback,
            validation_data=xtest,
            validation_steps=len(xtest))

Epoch 1/10
630/630 [==============================] - 138s 219ms/step - loss: 0.2389 - accuracy: 0.9363 - val_loss: 0.1424 - val_accuracy: 0.9787 - lr: 0.0010
Epoch 2/10
630/630 [==============================] - 137s 217ms/step - loss: 0.0248 - accuracy: 0.9929 - val_loss: 0.2145 - val_accuracy: 0.9764 - lr: 0.0010
Epoch 3/10
630/630 [==============================] - 136s 216ms/step - loss: 0.0182 - accuracy: 0.9947 - val_loss: 0.5314 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 4/10
630/630 [==============================] - 136s 215ms/step - loss: 0.0116 - accuracy: 0.9961 - val_loss: 0.4777 - val_accuracy: 0.9769 - lr: 0.0010
Epoch 5/10
630/630 [==============================] - 136s 216ms/step - loss: 0.0138 - accuracy: 0.9955 - val_loss: 0.7040 - val_accuracy: 0.9738 - lr: 0.0010
Epoch 6/10
630/630 [==============================] - 138s 219ms/step - loss: 0.0089 - accuracy: 0.9973 - val_loss: 0.2694 - val_accuracy: 0.9813 - lr: 0.0010
Epoch 7/10
630/630 [==========================

In [26]:
model.save('Sign.h5')

In [27]:
from skimage.transform import resize 
def detect(frame):
  img=resize(frame, (64,64,1))
  img= np.expand_dims (img,axis=0)
  if(np.max(img)>1):
    img=img/255.0
    prediction = model.predict(img)
    print(prediction)
    prediction = model.predict_classes(img) 
    print(prediction)
  

In [28]:
!tar -zcvf image_new.tgz Sign.h5

Sign.h5


In [29]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 11.1 MB/s eta 0:00:01


In [30]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                "url": "https://us-south.ml.cloud.ibm.com", 
                "apikey":"DX1QQrPIHs7H37jpSdZ-7ULbI7fYeZTLisIN5xyMkoY5"
}
client=APIClient(wml_credentials)

In [31]:
client=APIClient(wml_credentials)

In [32]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()  
    return(next(item for item in space['resources'] if item['entity']["name"]== space_name)['metadata']['id'])

In [33]:
space_uid=guid_from_space_name(client,'Newspace')
print("Space UID=" + space_uid)

Space UID=508e36ba-6bca-4094-93a1-1bcfda4638e8


In [34]:
client.set.default_space(space_uid)

'SUCCESS'

In [35]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [36]:
software_spec_uid= client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9") 
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [37]:
model_details=client.repository.store_model(model='image_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME: "CNN",
client.repository.ModelMetaNames. TYPE: "tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                           )
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [38]:
model_id

'a9fc068c-b143-4e7e-8eba-3267fb7342c9'

In [39]:
client.repository.download (model_id, 'mymodel.tar.gz')

Successfully saved model content to file: 'mymodel.tar.gz'


'/home/wsuser/work/mymodel.tar.gz'

In [40]:
from keras.models import load_model 
from keras.preprocessing import image

In [41]:
model=load_model("Sign.h5")

In [42]:
import numpy as np
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [72]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GMKt23iW2XdnrnutVPY4alkQgbgl5DYr-wBW2LYLge42',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-omd0vkwic10b6g'
object_key = 'Dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img=image.load_img

In [87]:
img=image.load_img(r"/home/wsuser/work/Dataset/test_set/A/90.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
y=np.argmax(model.predict(x),axis=1)
index=['A','B','C','D','E','F','G','H','I']
index[y[0]]

'A'

In [88]:
img=image.load_img( "/home/wsuser/work/Dataset/test_set/D/1.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
y=np.argmax(model.predict(x),axis = 1)
index=['A','B','C','D','E','F','G','H','I']
index[y[0]]

'D'

In [89]:
img=image.load_img(r"/home/wsuser/work/Dataset/test_set/G/1.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
y=np.argmax(model.predict(x), axis=1)
index=['A','B','C','D','E','F','G','H','I']
index[y[0]]

'G'